### source code from TFA

In [ ]:
# if cfg.MODEL.BACKBONE.FREEZE:
#     for p in self.backbone.parameters():
#         p.requires_grad = False
#     print("froze backbone parameters")

# if cfg.MODEL.PROPOSAL_GENERATOR.FREEZE:
#     for p in self.proposal_generator.parameters():
#         p.requires_grad = False
#     print("froze proposal generator parameters")

# if cfg.MODEL.ROI_HEADS.FREEZE_FEAT:
#     for p in self.roi_heads.box_head.parameters():
#         p.requires_grad = False
#     print("froze roi_box_head parameters")

### check cosine similarity bbox head

In [ ]:
from mmdet.models.utils import build_linear_layer
# torch.Size([1024, 256, 7, 7])
from torch import nn

a= nn.Linear((256,7,7), 7, bias=False)

fc_cls = build_linear_layer(
    cfg=None,
    in_features=256,
    out_features=7)
print(a)

### check gradient visualization

In [ ]:
from mmdet.models import build_detector
from mmcv import Config
config_path = "/home/dlsuncheng/FSOD/FsMMdet/Model/cos-ft/frcn_unfreeze_cos.py"

cfg = Config.fromfile(config_path)
model = build_detector(
    cfg.model,
    train_cfg=cfg.get('train_cfg'),
    test_cfg=cfg.get('test_cfg'))
model.init_weights()
for tag, value in model.backbone.named_parameters():  
    print(tag)
    print(value.grad)  

### visualize NEU_DET

In [ ]:
# !python /home/dlsuncheng/FSOD/FsMMdet/MMdet/tools/misc/browse_dataset.py /home/dlsuncheng/FSOD/FsMMdet/Model/cos-ft/frcn_pretrain.py \
#     --output-dir /home/dlsuncheng/Dataset/NEU-DET/VOC2012/Visualize/ 

### coco post analysis

#### 1. 总体精度        
#### 2. 是否存在错误分类  
#### 3. 是否存在漏检
#### 4. 结果可视化

In [1]:
# pth_path = "/home/dlsuncheng/Work_dir/Steel_Defect/20211021/FRCN_all/epoch_21.pth"
import json
import pandas as pd
from collections import Counter
import numpy as np
from sklearn.metrics import confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt
def Read_Json(path):
    with open(path,"r") as load_f:
        json_file = json.load(load_f)
    return json_file

result_path = "./Result/FRCN_all_test.bbox.json"
test_anno_path = "/home/dlsuncheng/FSOD/FsMMdet/Datasets/NEU_DET/COCO_Annotation/test.json"
result_json = Read_Json(result_path)
test_json = Read_Json(test_anno_path)

img_info = pd.DataFrame(columns=["image_id","gt_category","pred_category"])
gt_image_id_list = [img_info["image_id"] for img_info in test_json["annotations"]]
gt_image_id = list(set(gt_image_id_list))
gt_image_id.sort()
img_info["image_id"] = gt_image_id

## 误分类分析
### 同一张图片内的类别应该相同
### image_id:[category_ids]
pred_category = []
gt_category = []
for img_id in gt_image_id:
    pred_cat_id_list = np.array([img["category_id"] for img in result_json if img["image_id"]==img_id])
    gt_cat_id_list = np.array([anno["category_id"] for anno in test_json["annotations"] if anno["image_id"]==img_id])
    gt_cat_id = gt_cat_id_list[0]*np.ones(pred_cat_id_list.shape)
    pred_category.extend(pred_cat_id_list)
    gt_category.extend(gt_cat_id)

con=confusion_matrix(gt_category,pred_category)
# sn.set()

# plt.rcParams['font.sans-serif']=['SimHei']
# fig = plt.figure(figsize=(10,10))
# sn.heatmap(con,annot = True,linewidths=1)
# plt.savefig("./cls_heat.png")
# plt.show()
### 是否存在漏检
# pred_image_id_list = [img_info["image_id"] for img_info in result_json]
# pred_bbox_num = dict(Counter(pred_image_id_list))
# gt_image_id_list = [img_info["image_id"] for img_info in test_json["annotations"]]
# gt_bbox_num = dict(Counter(gt_image_id_list))

# pred_num = list(pred_bbox_num.values())
# gt_num = list(gt_bbox_num.values())
# for index in range(len(pred_num)):
#     if pred_num[index]-gt_num[index]<0:
#         print(1)

## 没有漏检，但是部分图片多出来了很多bbox

### 权重——去除最后一个分类层，和其他多余信息

In [9]:
# parser.add_argument('--param-name', type=str, nargs='+',
#         default=['roi_heads.box_predictor.cls_score',
#                     'roi_heads.box_predictor.bbox_pred'],
#         help='Target parameter names')

# if args.method == 'remove':
#     for param_name in args.param_name:
#         del ckpt['model'][param_name + '.weight']
#         if param_name+'.bias' in ckpt['model']:
#             del ckpt['model'][param_name+'.bias']
#     save_ckpt(ckpt, save_path)
#     return

import torch
pth_path = "/home/dlsuncheng/Work_dir/Steel_Defect/20211021/FRCN_cos-ft/epoch_24.pth"
def reset_ckpt(ckpt):
    if 'scheduler' in ckpt:
        del ckpt['scheduler']
    if 'optimizer' in ckpt:
        del ckpt['optimizer']
    if 'iteration' in ckpt:
        ckpt['iteration'] = 0

def save_ckpt(ckpt, save_name):
    torch.save(ckpt, save_name)
    print('save changed ckpt to {}'.format(save_name))

ckpt = torch.load(pth_path)

param_name = 'roi_head.bbox_head'

for parameter in list(ckpt["state_dict"].keys()):
    if "fc_cls" in parameter or "fc_reg" in parameter:
        # del ckpt['state_dict'][parameter]
        print(parameter)
# reset_ckpt(ckpt)

del ckpt["meta"],ckpt["optimizer"]

roi_head.bbox_head.fc_cls.weight
roi_head.bbox_head.fc_cls.bias
roi_head.bbox_head.fc_reg.weight
roi_head.bbox_head.fc_reg.bias


In [17]:
import torch
tfa_cos_path = "/home/dlsuncheng/FSOD/FsMMdet/Weights/tfa_cos.pth"
tfa_fc_path = "/home/dlsuncheng/FSOD/FsMMdet/Weights/tfa_fc.pth"

tfa_cos_pth = torch.load(tfa_cos_path)
tfa_fc_pth = torch.load(tfa_fc_path)
print([param_name for param_name in tfa_cos_pth["model"].keys() if "roi_heads" in param_name])
print([param_name for param_name in tfa_fc_pth["model"].keys() if "roi_heads" in param_name])

['roi_heads.box_head.fc1.weight', 'roi_heads.box_head.fc1.bias', 'roi_heads.box_head.fc2.weight', 'roi_heads.box_head.fc2.bias', 'roi_heads.box_predictor.cls_score.weight', 'roi_heads.box_predictor.bbox_pred.weight', 'roi_heads.box_predictor.bbox_pred.bias']
['roi_heads.box_head.fc1.weight', 'roi_heads.box_head.fc1.bias', 'roi_heads.box_head.fc2.weight', 'roi_heads.box_head.fc2.bias', 'roi_heads.box_predictor.cls_score.weight', 'roi_heads.box_predictor.cls_score.bias', 'roi_heads.box_predictor.bbox_pred.weight', 'roi_heads.box_predictor.bbox_pred.bias']


['roi_heads.box_head.fc1.weight', 'roi_heads.box_head.fc1.bias', 'roi_heads.box_head.fc2.weight', 'roi_heads.box_head.fc2.bias', 'roi_heads.box_predictor.cls_score.weight', 'roi_heads.box_predictor.bbox_pred.weight', 'roi_heads.box_predictor.bbox_pred.bias']
